</br>

# Sampleプログラム：1次元水平壁内熱水分同時移動解析  
version 2.0 1次元熱水分同時移動計算を行う

※2021/01/12 加筆・修正  
初めて本プログラムを使う際は以下のセルをMarkdownからCodeに変更し、CSVおよびDataFramesパッケージをインストールすること

using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")

## 手順0：計算に必要なモジュールを読み込む  

In [65]:
using Dates # 解析内時間を計算するためのモジュール
include("./module/cell.jl"); # 多孔質材料の構成要素を表すモジュール
include("./module/air.jl");  # 空間の構成要素を表すモジュール
include("./module/room.jl"); # 室空間の構成要素を表すモジュール
include("./module/climate.jl"); # 気象データを取り扱うモジュール
include("./module/boundary_condition.jl"); # 境界条件を表すモジュール
include("./module/transfer_in_media.jl"); # 材料中における熱・水分の移動を表すモジュール
include("./logger.jl"); # 計算結果を保存するモジュール

## 手順１：解析対象モデルの構築

### 1.1 壁の構築
Cell構造体から構成される配列であるwall（壁）を作成する。  
Cell構造体は空間情報、熱力学的状態量、材料名称の主に3つを有し、これらの情報から各物性値を求めることができる。  
Cell構造体の詳細はcell.ipynbのファイルを確認すること。  
wallの作成方法は、主に下記の3つがある。  
1. 手動で壁の各コントールボリュームの幅、温度、相対湿度、材料名を記述する方法
2. 壁の幅と分割数、初期の温度、相対湿度、材料名を記入し、コントロールボリュームを自動で作成する方法
3. ./input_data/wall_data内に格納されたcsvファイルから各コントロールボリュームの情報を読み込む方法

ここでは最も一般的な1.の方法でwallを作成する。

In [66]:
# CSVファイル読込用モジュール
using CSV, DataFrames
include("./module/property_conversion.jl")

# 壁データ読込関数
function input_cell_data(file_name::String)
    file_directory = "./input_data/wall_data/"*string(file_name)*".csv"
    input_data = CSV.File(file_directory, header = 1) |> DataFrame
    cell = [Cell() for i = 1:length(input_data.i)]
    
    for i = 1:length(input_data.i)
        cell[i].i = [input_data.i[i], 1, 1]
        cell[i].dx = input_data.dx[i]
        cell[i].temp = input_data.temp[i] + 273.15
        cell[i].material_name = input_data.material_name[i]
        cell[i].rh = input_data.rh[i] / 100
        cell[i].miu = convertRH2Miu(temp = cell[i].temp, rh = cell[i].rh)
        
        # dx2設定
        if i == 1 || i == length(input_data.i)
            cell[i].dx2 = cell[i].dx
        else
            cell[i].dx2 = cell[i].dx / 2
        end
    end
    return cell
end

input_cell_data (generic function with 2 methods)

In [67]:
# L個のCell構造体から構成されるwallを作成
# L = 6
# width = 0.005
# wall = [ Cell() for i = 1 : L ];

# CSVファイルから壁構成を読込
wall = input_cell_data("storage_shed_optimal")
L = length(wall)

# 壁厚の確認表示
total_width = sum([cell.dx for cell in wall])
println("総壁厚: ", total_width*1000, "mm")
println("セル数: ", L, "個")
for i = 1:L
    println("Layer ", i, ": ", wall[i].material_name, " (", wall[i].dx*1000, "mm)")
end

総壁厚: 63.0mm
セル数: 6個
Layer 1: metal_plate (0.5mm)
Layer 2: glass_wool_16K (16.7mm)
Layer 3: glass_wool_16K (16.7mm)
Layer 4: glass_wool_16K (16.6mm)
Layer 5: plasterboard (6.25mm)
Layer 6: plasterboard (6.25mm)


In [68]:
# 各CVの情報を入力する。なお、i,dx,dx2,temp,miu（水分化学ポテンシャル）,material_nameは入力必須項目である。
# for i = 1 : L
#     wall[i].i = [ i, 1, 1 ]
#     if i == 1 || i == L     wall[i].dx  = ( width / ( L - 1 ) / 2 )
#     else                    wall[i].dx = width / ( L - 1 ) end
#     if i == 1               wall[i].dx2 = 0.0
#     elseif i == L           wall[i].dx2 = wall[i].dx2
#     else                    wall[i].dx2 = wall[i].dx / 2 end
#     wall[i].temp = 10.0 + 273.15
#     wall[i].rh = 0.6
#     wall[i].miu = convertRH2Miu( temp = wall[i].temp, rh = wall[i].rh )
#     wall[i].material_name = "mud_wall"
# end

### 1.2 境界条件と空間の構成要素の設定

Air構造体を作成する。  
Air構造体は空間情報（体積）、熱力学的状態量、伝達係数の主に3つを有する。  
Air構造体の詳細はair.ipynbのファイルを確認すること。  
Air構造体は手動で情報を入力する必要があり、

壁は表面（配列の1側）、裏面（配列のend側）ともに空気に接するものとする。  
このとき、表面側を室内側、裏面側を外気とし以下のように設定する。  

- air_in （室内側空気）：対流熱伝達率4.9
- air_out（外気側空気）：対流熱伝達率18.6  

なお、初期の温度、相対湿度に関しては測定した気象データを用いるため適当な値とした。

In [69]:
air_in = BC_Robin()
air_in.air.name = "indoor"      # 名称
air_in.air.temp = 10.0 + 273.15 # 温度
air_in.air.rh   = 0.7           # 相対湿度
air_in.alphac = 4.9             # 対流熱伝達率
air_in.alphar = 4.4             # 放射熱伝達率
air_in.alpha  = air_in.alphac + air_in.alphar   # 総合熱伝達率
air_in.aldm   = aldm_by_alphac(air_in)          # 湿気伝達率（ルイス関係より算出）

2.8834097223745298e-8

In [70]:
air_out = BC_Robin()
air_out.air.name = "outdoor"     # 名称
air_out.air.temp = 20.0 + 273.15 # 温度
air_out.air.rh   = 0.6           # 相対湿度
air_out.alphac = 18.6            # 対流熱伝達率
air_out.alphar = 4.4             # 放射熱伝達率
air_out.alpha  = air_out.alphac + air_out.alphar   # 総合熱伝達率（23.0 W/m²K）
air_out.aldm   = aldm_by_alphac(air_out)          # 湿気伝達率（ルイス関係より算出）

1.0571823166213907e-7

### 1.3 解析対象モデルの作成（壁と境界条件の結合）
1.1および1.2にて作成した壁および境界条件を結合し、解析対象モデルとする。

In [71]:
target_model = vcat(air_in, wall, air_out)

8-element Vector{Any}:
 BC_Robin("NoName", Air(0, "indoor", 0.0, 0.0, 0.0, 0.0, 283.15, 0.7, 0.0, 0.0, 101325.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), Cell([1, 1, 1], [0.0, 0.0, 0.0], 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 101325.0, "NoName", [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]), 0.0, 0.0, 0.0, 0.0, 9.3, 4.9, 4.4, 2.8834097223745298e-8, Dict("LATIl" => 360.0, "e" => 0.0, "ALTIs" => -180.0, "LATIs" => -180.0, "a" => 0.0, "ALTIl" => 360.0), 0.0, 0.0)
 Cell([1, 1, 1], [0.0, 0.0, 0.0], 0.0005, 1.0, 1.0, 0.0005, 1.0, 1.0, 283.15, -46647.31840378666, 0.7, 0.0, 0.0, 101325.0, "metal_plate", [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]])
 Cell([2, 1, 1], [0.0, 0.0, 0.0], 0.0167, 1.0, 1.0, 0.00835, 1.0, 1.0, 283.15, -46647.31840378666, 0.7, 0.0, 0.0, 101325.0, "glass_wool_16K", [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0], [0.0, 0.0,

## 手順２：流量計算式および収支式を構築する

流量計算式および収支計算式は一般的な第三種境界条件とする。  
日射や降雨等を考慮せず、水平な壁の場合は以下のような式となる。 
この部分は必要な計算に応じて適宜修正すること。

In [72]:
function cal_new_temp_miu_neumann( target_model, dt )
    ###################################
    # 壁体内部の流量計算
    q =  [ cal_q( target_model[i], target_model[i+1] ) for i = 1:length(target_model)-1 ]
    jv = [ cal_jv( target_model[i], target_model[i+1] ) for i = 1:length(target_model)-1 ]
    jl = [ cal_jl( target_model[i], target_model[i+1], 0.0 ) for i = 1:length(target_model)-1 ]

    ###################################
    # 熱・水分の収支計算
    ntemp = [ cal_newtemp( target_model[i+1], q[i] - q[i+1], - ( jv[i] - jv[i+1] ), dt ) for i = 1:length(target_model)-2 ]
    nmiu  = [ cal_newmiu(  target_model[i+1], jv[i] - jv[i+1], jl[i] - jl[i+1], dt ) for i = 1:length(target_model)-2 ]    
    
    return ntemp, nmiu
end

cal_new_temp_miu_neumann (generic function with 1 method)

## 手順３：計算条件に関する設定  


計算条件として、主に下記の設定を行う。
- dt : 計算の時間刻み 
- date : 計算開始時刻
- end_date : 計算終了時刻  

なお、dtは実数（小数点以下も可）、dateはJuliaのDateTimeを使って設定すること。

In [ ]:
# 計算の時間刻み
dt = 5.0

# 計算開始時刻
calc_date = DateTime(2012,4,1,0,0,0)

# 計算終了時刻
end_date = DateTime(2012,4,8,00,00,00)

2012-04-08T00:00:00

## 手順４：外界気象条件に関する設定  

外界気象の扱い方の詳細はclimate.ipynbに示してあるため、詳細はそちらを参照のこと。

読み込み方は、関数input_climate_data()を用い、input_data/climate_data内にある気象データ（CSV形式）のディレクトリを指定する。  
ヘッダー位置（header）はデフォルトで3行目としてあるため、変更が必要な場合記載すること。 
読み込んだ気象データはClimate型として保存される。  

※1 読み込めるのはcsvファイルだけであるためExcelファイルは読み込めない。  
※2 入力される温度・相対湿度は「℃」セルシウス度、「％」の単位を想定している。  
温度をケルビンあるいは相対湿度を無次元「－」で利用する場合は要注意のこと。

### 室内側環境条件の読み込み

In [74]:
climate_data_in  = input_climate_data(
    file_name = "./input_data/climate_data/sample_climate_data.csv", 
    header = 3 );

climate_data_in.air = air_in.air

# ロギングインターバルの確認
climate_data_in.logging_interval

60

### 外気側環境条件の読み込み

In [75]:
climate_data_out = input_climate_data( 
    file_name = "./input_data/climate_data/sample_climate_data.csv", 
    header = 3);

climate_data_out.air = air_out.air

# ロギングインターバルの確認
climate_data_out.logging_interval

60

## 手順５：計算結果の保存

計算結果の保存方法はlogger.ipynbファイルに記載してある。  

基本的な手順としては、open関数を用いて計算結果を保存するファイルを開き、print関数を用いて計算結果をファイルに書き込むという手順である。  

In [76]:
# ロガーの設定
logger_room = set_logger("result_storage_shed", 10.0, ["temp","rh","ah"], target_model)
# ヘッダーの書き込み
write_header_to_logger( logger_room )

# 初期値の書き込み
write_data_to_logger( logger_room, calc_date )

## 手順６：繰り返し（ループ）計算  

ループ計算は手順３で設定した終了時刻まで計算を行うようなプログラムである。  

In [77]:
# 計算回数の設定
while calc_date ≠ end_date
    
    # １：環境データの更新
    reset_climate_data( climate_data_in )
    reset_climate_data( climate_data_out )

    # ２：新値の計算
    ntemp, nmiu = cal_new_temp_miu_neumann( target_model, dt )
    
    # ３：値の上書き
    for i = 1:length(wall)
        wall[i].temp = ntemp[i]
        wall[i].miu  = nmiu[i]
    end
    
    # 計算時刻の表示
    if hour(calc_date) == 0 && minute(calc_date) == 0 && second(calc_date) == 0 && millisecond(calc_date) == 0;
        println(Dates.format(calc_date, "yyyy/mm/dd HH:MM"), " 外気：温度" *  string( round(air_out.air.temp -273.15, digits = 1 ) ) * "[℃]" * " 湿度" * string( round( air_out.air.rh, digits = 2 ) ) * "[-]" )
    end
        
    # ４：時間経過の計算
    calc_date = calc_date + Millisecond(dt*1000)
    climate_data_in.date    = calc_date
    climate_data_out.date   = calc_date
    
    # ５：データのロギング  
    if mod(minute(calc_date), 5) == 0 && second(calc_date) == 0 && millisecond(calc_date) == 0
        write_data_to_logger( logger_room, calc_date )
    end
    
end

2012/04/01 00:00 外気：温度3.7[℃] 湿度0.77[-]
2012/04/02 00:00 外気：温度6.7[℃] 湿度0.68[-]
2012/04/03 00:00 外気：温度17.1[℃] 湿度0.63[-]
2012/04/04 00:00 外気：温度8.6[℃] 湿度0.39[-]
2012/04/05 00:00 外気：温度11.5[℃] 湿度0.5[-]
2012/04/06 00:00 外気：温度14.1[℃] 湿度0.45[-]
2012/04/07 00:00 外気：温度11.0[℃] 湿度0.38[-]


In [78]:
for file in logger_room.file
    close(file)
end

In [79]:
calc_date

2012-04-08T00:00:00